In [33]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from googlesearch import search
from openpyxl import load_workbook

def extract_emails_from_url(url):
    """Extract emails from a webpage."""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            page_content = response.text
            # Find all email patterns in the page content
            emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', page_content)
            return set(emails)  # Return unique emails
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
    return []

def search_emails(company_name):
    """Search for emails by company name."""
    query = f"{company_name} email"
    emails_found = set()
    for url in search(query, num_results=5):  # Adjust `num_results` for more pages
        emails = extract_emails_from_url(url)
        emails_found.update(emails)
    return list(emails_found)

def main():
    input_file = "input.xlsx"  # Replace with your input file name
    output_file = "outputtt.xlsx"  # Replace with your desired output file name

    # Load the input Excel file into a DataFrame
    data = pd.read_excel(input_file, sheet_name=0)
    company_column = "name"  # Assuming column named "name" contains company names

    # Create the output file with the initial DataFrame
    data['Emails'] = None  # Add an Emails column if it doesn't exist
    data.to_excel(output_file, index=False, engine="openpyxl")

     # Load the output file with openpyxl for live updates
    workbook = None
    try:
        # Load the output file with openpyxl for live updates
        workbook = load_workbook(output_file)
        sheet = workbook.active

        # Process each company
        for index, row in data.iterrows():
            if(index<322):
                print("skipping")
                continue

            company_name = row[company_column]
            print(f"Searching for emails for company: {company_name}")
            
            # Fetch emails for the company
            emails = search_emails(company_name)
            email_str = ", ".join(emails) if emails else "No emails found"
            
            # Update the Excel sheet directly
            sheet.cell(row=index + 2, column=3).value = email_str  # Assuming Emails is the 3rd column
            print(f"Updated: {company_name} : {email_str}")

            # Save the workbook after each update
            try:
                workbook.save(output_file)
                print(f"File updated: {output_file}")
            except PermissionError:
                print("File is currently open. Please save and close it before retrying.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Ensure the workbook is closed properly
        if workbook:
            try:
                workbook.close()
                print("Workbook closed safely.")
            except Exception as e:
                print(f"Failed to close the workbook: {e}")

    print("Email extraction complete.")

main()


PermissionError: [Errno 13] Permission denied: 'outputtt.xlsx'